<a href="https://colab.research.google.com/github/amaillis94/project2_nyc_classification/blob/main/NYC_analysis_amaillis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv("/content/.env")
# Confirming .env
print("User:", os.getenv("DB_USER"))

In [ ]:
#Connect to Aiven to source data
from sqlalchemy import create_engine


engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}?sslmode=require",
    connect_args={
        "sslmode": "require",
        "sslrootcert": "/content/ca.pem"
    }
)


In [ ]:
nyc_data=pd.read_sql("SELECT * FROM df;", engine)
nyc_data.head(5)

In [ ]:
nyc_data.info()

In [ ]:
#drop columns that are either entirely or mostly (>90%) missing
nyc_data.dropna(axis=1, thresh=(len(nyc_data)*.1), inplace=True)
nyc_data.shape

In [ ]:
#Make all datetimes, datetime format
nyc_data['Created Date'] = pd.to_datetime(nyc_data['Created Date'])
nyc_data['Closed Date'] = pd.to_datetime(nyc_data['Closed Date'])
nyc_data['Due Date'] = pd.to_datetime(nyc_data['Due Date'])

In [ ]:
top_n = 5
top_complaints = nyc_data['Complaint Type'].value_counts().nlargest(top_n)
print(f"\nTop {top_n} Complaint types:")
print(top_complaints)

In [ ]:
nyc_data['hrs_to_completion'] = (nyc_data['Closed Date'] - nyc_data['Created Date']).dt.total_seconds() / 3600
nyc_data['hrs_to_completion'].describe()

In [ ]:
#Plot distribution of SR hrs to completion
#Max hrs_to_completion=592.8 omitted from figure to more easily visualize distr. of variable

nyc_data['hrs_to_completion'].hist(bins=250,range=[0,100])

In [ ]:
nyc_data['month']=nyc_data['Created Date'].dt.month
nyc_data['year']=nyc_data['Created Date'].dt.year

In [ ]:
grouped_counts = nyc_data.groupby(['month','year','Borough']).size()
print(grouped_counts)

In [ ]:
#remove unspecified rows? Or map them to a Borough using lat/long?
#Update: remove. Lat/Long missing for borough='unspecified'
filtered_df=nyc_data[nyc_data['Borough']=='Unspecified']
filtered_df.shape
print(filtered_df.info())

nyc_data = nyc_data[nyc_data.Borough != 'Unspecified']


In [ ]:
#Daily average number of service requests by borough
index = nyc_data.set_index('Created Date')

daily_counts = index.groupby('Borough').resample('D').size().unstack(fill_value=0)
average_daily_count = daily_counts.mean(axis=1)

print("\nAverage daily number of service requests per Borough:")
print(average_daily_count)

In [ ]:
#Create figure to check counts of complaint by Borough
nyc_data['hour'] = nyc_data['Created Date'].dt.hour
hr_counts = nyc_data.groupby(['Borough', 'hour']).size().reset_index(name='hr_count')

# Pivot to get borough as columns
pivot_df = hr_counts.pivot(index='hour', columns='Borough', values='hr_count').fillna(0)

# Set up bar positions
x = np.arange(len(pivot_df.index))
width = 0.15
fig, ax = plt.subplots(figsize=(12,6))


for i, borough in enumerate(pivot_df.columns):
    ax.bar(x + i*width, pivot_df[borough], width, label=borough)

# Plot
ax.set_ylabel('Service request count')
ax.set_title('Service requests by hour per Borough')
ax.set_xticks(x + width * (len(pivot_df.columns)-1) / 2, pivot_df.index)
ax.legend(title="Borough")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Creating boolean weekend variable
nyc_data['day_int'] = nyc_data['Created Date'].dt.dayofweek
nyc_data['is_weekend'] = nyc_data['day_int'] > 4

In [ ]:
#Set up data limited to only top 5 complaints
top5_complaints = nyc_data['Complaint Type'].value_counts().nlargest(5).index
top5 = nyc_data[nyc_data['Complaint Type'].isin(top5_complaints)]

exploratory = (top5.groupby(["is_weekend", "Borough", "Complaint Type"]).size().reset_index(name="event_count"))

#Visualize top 5 complaints by Borough and weekend vs. weekday
sea.catplot(
    data=exploratory,
    x="is_weekend",
    y="event_count",
    hue="Complaint Type",
    col="Borough",
    kind="bar",
    errorbar=None
)
plt.show()

In [ ]:
descriptive_stats = top5.groupby('Complaint Type')['hrs_to_completion'].describe()
print("Descriptive statistics for Hourly completion time grouped by Complaint Type:")
print(descriptive_stats)

In [ ]:
descriptive_stats1 = top5['hrs_to_completion'].describe()
print("Descriptive statistics for overall Hourly completion time:")
print(descriptive_stats1)

In [ ]:
#Outcome variable (>5 hrs to completion Y/N)
nyc_data['y1'] = nyc_data['hrs_to_completion'].apply(lambda x: 1 if x > 4.999 else 0)

print(pd.crosstab(nyc_data['y1'], nyc_data['hrs_to_completion']))